# Dubinska analiza podataka - projektni zadatak

## 1. Predprocesiranje podataka

In [140]:
phoneData <- read.csv('phone_specs.csv')

str(phoneData)

'data.frame':	1359 obs. of  22 variables:
 $ Index                 : int  0 1 2 3 4 5 6 7 8 9 ...
 $ Name                  : chr  "OnePlus 7T Pro McLaren Edition" "Realme X2 Pro" "iPhone 11 Pro Max" "iPhone 11" ...
 $ Brand                 : chr  "OnePlus" "Realme" "Apple" "Apple" ...
 $ Model                 : chr  "7T Pro McLaren Edition" "X2 Pro" "iPhone 11 Pro Max" "iPhone 11" ...
 $ Battery.capacity..mAh.: int  4085 4000 3969 3110 4000 3800 4085 4300 6000 4000 ...
 $ Screen.size..inches.  : num  6.67 6.5 6.5 6.1 6.4 6.55 6.67 6.8 6.59 6.39 ...
 $ Touchscreen           : chr  "Yes" "Yes" "Yes" "Yes" ...
 $ Resolution.x          : int  1440 1080 1242 828 1080 1080 1440 1440 1080 1080 ...
 $ Resolution.y          : int  3120 2400 2688 1792 2340 2400 3120 3040 2340 2340 ...
 $ Processor             : int  8 8 6 6 8 8 8 8 8 8 ...
 $ RAM..MB.              : int  12000 6000 4000 4000 6000 8000 8000 12000 8000 6000 ...
 $ Internal.storage..GB. : num  256 64 64 64 128 128 256 256 128 128 .

In [141]:
table(phoneData$Operating.system)


   Android BlackBerry   Cyanogen        iOS   Sailfish      Tizen    Windows 
      1299         10         10         17          1          3         19 

In [143]:
require(tidyr)

phoneData <- phoneData %>%
    filter(!(Operating.system == 'Sailfish' | Operating.system == 'Tizen' |
            Operating.system == 'Cyanogen' | Operating.system == 'BlackBerry')) %>%    # delete rows with less know OS (only 24 rows)
    mutate(across(c(Touchscreen, Wi.Fi, Bluetooth, GPS, X3G, X4G..LTE),
          ~as.numeric(ifelse(.x == 'Yes', 1, 0)))) %>%                        # convert categorical Yes/No to 1/0
    mutate(value = 1) %>% spread(Operating.system, value,  fill = 0 ) %>%     # One-hot-encoding OS column
    mutate(across(c(5,6, 8:14, 18), scale)) %>%                               # normalization with scale() for numeric data
    mutate(Price=round(Price/87.95)) %>%                                      # convert indian rupee to euro
    mutate(PriceCategory = ifelse(Price <= 400, 'Low-End', ifelse(Price <= 700, 'Mid-Range', 'High-End')))

## 2. Klasifikacija - umjetne neuronske mreže

## 3. Ansambli - slučajne šume

## 4. Asocijacijska analiza - učenje asocijacijskih pravila

## 5. Grupiranje k-sredina